In [1]:
!apt-get update -y

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,551 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [4]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [6]:
!pip install -q findspark
import findspark
findspark.init()

In [1]:
! pip install pyspark
!pip install gdown

In [1]:
import gdown

def donwlod_file(url, file_name):
    gdown.download(url, file_name, quiet=True, use_cookies=False)

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Window
from pyspark.sql.functions import col, udf


spark = (SparkSession
.builder
.appName("deep")
.getOrCreate())

In [2]:
sc = spark.sparkContext

In [3]:
path = r"C:\Users\anude\Downloads\projects\internship-tasks\chess\chess_wc_history_game_info.csv"

df = spark.read.csv(path, header=True, sep=",").cache()

In [4]:
# Create dataframes for each of the datasets. Give proper column names and datatypes. Include basic schema validation. (Refer the schema provided
# with the data for reference) Check out spark.read.format from here to create spark dataframe.

df.createOrReplaceTempView("history_game_info")
df = spark.sql("SELECT * FROM history_game_info")
df.show()

+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event|      site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021| Dubai UAE| 2021.11.26|    1|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|     draw|      null|            dra

In [5]:
df.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- game_order: string (nullable = true)
 |-- event: string (nullable = true)
 |-- site: string (nullable = true)
 |-- date_played: string (nullable = true)
 |-- round: string (nullable = true)
 |-- white: string (nullable = true)
 |-- black: string (nullable = true)
 |-- result: string (nullable = true)
 |-- white_elo: string (nullable = true)
 |-- black_elo: string (nullable = true)
 |-- white_title: string (nullable = true)
 |-- black_title: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- winner_elo: string (nullable = true)
 |-- loser: string (nullable = true)
 |-- loser_elo: string (nullable = true)
 |-- winner_loser_elo_diff: string (nullable = true)
 |-- eco: string (nullable = true)
 |-- date_created: string (nullable = true)
 |-- tournament_name: string (nullable = true)



In [6]:
df1 = spark.read.csv(r"C:\Users\anude\Downloads\projects\internship-tasks\chess\chess_wc_history_moves.csv", header=True, sep=",")

In [7]:
df1.createOrReplaceTempView("history_moves")
df1 = spark.sql("SELECT * FROM history_moves")
df1.show()

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [8]:
df2 = spark.read.csv(r"C:\Users\anude\Downloads\projects\internship-tasks\chess\eco_codes.csv", header=True, sep=",")

In [9]:

df2.createOrReplaceTempView("eco_codes")
df2 = spark.sql("SELECT * FROM eco_codes")
df2.show()

+---+--------------------+--------------------+--------+---------+
|eco|            eco_name|         eco_example|eco_type|eco_group|
+---+--------------------+--------------------+--------+---------+
|A00|    Uncommon Opening|  1 g4, a3, h3, etc.|       A|    Flank|
|A01|Nimzovich-Larsen ...|                1 b3|       A|    Flank|
|A02|      Bird's Opening|                1 f4|       A|    Flank|
|A03|      Bird's Opening|             1 f4 d5|       A|    Flank|
|A04|        Reti Opening|               1 Nf3|       A|    Flank|
|A05|        Reti Opening|           1 Nf3 Nf6|       A|    Flank|
|A06|        Reti Opening|            1 Nf3 d5|       A|    Flank|
|A07|King's Indian Attack|       1 Nf3 d5 2 g3|       A|    Flank|
|A08|King's Indian Attack|1 Nf3 d5 2 g3 c5 ...|       A|    Flank|
|A09|        Reti Opening|       1 Nf3 d5 2 c4|       A|    Flank|
|A10|             English|                1 c4|       A|    Flank|
|A11|English, Caro-Kan...|             1 c4 c6|       A|    Fl

 1. List of Winners of Each World champions Trophy Hint: Total Result of all rounds of Tournament for that player is considered as that player's
 Score/Result.
 Result attributes: winner, tournament_name

In [10]:
# winner = spark.sql("SELECT winner, tournament_name FROM history_game_info")
# winner.show()

l_data = df.filter(col("winner") != "draw").select("winner", "tournament_name").drop_duplicates(["winner", "tournament_name"])
l_data.show()
l_data.toPandas().to_csv("df1.csv")

+------------------+---------------+
|            winner|tournament_name|
+------------------+---------------+
|          Bareev,E|  FideChamp2002|
|           Dreev,A|  FideChamp2002|
|    Andersson, Ulf|  FideChamp1999|
|    Svidler, Peter|  FideChamp1998|
|         Ehlvest,J|  FideChamp2002|
|         Delchev,A|  FideChamp2004|
|      Ponomariov,R|  FideChamp2002|
|      Zvjaginsev,V|  FideChamp2002|
| Steinitz, William| WorldChamp1890|
|          Kudrin,S|  FideChamp2004|
|Istratescu, Andrei|  FideChamp1998|
|          Sakaev,K|  FideChamp2002|
|        Ciuksyte,D|  FideChamp2002|
|         Lautier,J|  FideChamp2002|
|      Psakhis, Lev|  FideChamp1999|
|     Dreev, Alexey|  FideChamp1999|
|         Kramnik,V| WorldChamp2006|
|     Lautier, Joel|  FideChamp1999|
|           Anand,V| WorldChamp2010|
|        Nakamura,H|  FideChamp2004|
+------------------+---------------+
only showing top 20 rows



2. List of Players with number of times they have won Tournament in descending order(Max to min).
Result attributes: player_name, number_of_wins

In [11]:
# player_wins = spark.sql("SELECT winner as player_name, count(winner) as number_of_wins FROM history_game_info GROUP BY winner ORDER BY number_of_wins DESC")
# player_wins.show()

no_of_times = df.filter(col("winner") != "draw").select("winner", "tournament_name").groupBy("winner").count().orderBy("count", ascending=False)
no_of_times.show()
no_of_times.toPandas().to_csv("df2.csv")

+--------------------+-----+
|              winner|count|
+--------------------+-----+
|     Lasker, Emanuel|   52|
|     Karpov, Anatoly|   47|
|  Botvinnik, Mikhail|   46|
|   Steinitz, William|   43|
| Alekhine, Alexander|   43|
|           Topalov,V|   32|
|      Kasparov, Gary|   31|
|             Anand,V|   30|
|    Smyslov, Vassily|   24|
|            Adams,Mi|   20|
|      Kasimdzhanov,R|   19|
|          Grischuk,A|   17|
|           Svidler,P|   16|
|      Adams, Michael|   16|
|            Bareev,E|   15|
|            Shirov,A|   15|
|             Dreev,A|   15|
|   Chigorin, Mikhail|   14|
|Khalifman, Alexander|   14|
|           Euwe, Max|   14|
+--------------------+-----+
only showing top 20 rows



3. Most and Least Popular eco move in world championship history.
Result attributes: eco, eco_name, number_of_occurences
Final result will have only two rows

In [12]:
eco = spark.sql("(SELECT h.eco, e.eco_name, count(h.eco) as number_of_occurences FROM history_game_info h JOIN eco_codes e ON h.eco = e.eco GROUP BY h.eco, e.eco_name ORDER BY number_of_occurences DESC LIMIT 1) UNION ALL (SELECT h.eco, e.eco_name, count(h.eco) as number_of_occurences FROM history_game_info h JOIN eco_codes e ON h.eco = e.eco GROUP BY h.eco, e.eco_name ORDER BY number_of_occurences ASC LIMIT 1)")
eco.show()
eco.toPandas().to_csv("df3.csv")

+---+--------------+--------------------+
|eco|      eco_name|number_of_occurences|
+---+--------------+--------------------+
|C42|Petrov Defense|                  64|
|C46| Three Knights|                   1|
+---+--------------+--------------------+



4. Find the eco move with most winnings.
Ps. Use this opening move in your next chess game
Result attributes: eco, eco_name


In [13]:
eco_wins = spark.sql("SELECT h.eco, e.eco_name, count(winner) as number_of_wins FROM history_game_info h JOIN eco_codes e ON h.eco = e.eco WHERE h.result = '1-0' GROUP BY h.eco, e.eco_name ORDER BY number_of_wins DESC LIMIT 1")
eco_wins.show()
eco_wins.toPandas().to_csv("df4.csv")

+---+-----------------+--------------+
|eco|         eco_name|number_of_wins|
+---+-----------------+--------------+
|B90|Sicilian, Najdorf|            19|
+---+-----------------+--------------+



5. Longest and shortest game ever played in a world championship in terms of move.
Chess Funda: "move" is completed once both White and Black have played one turn. e.g If a game lasts 10 moves, both White and Black have
played 10 moves)
Result attributes: game_id, event, tournament_name, number_of_moves
Final result will have only two rows

In [14]:
longest_shortest = spark.sql("(SELECT h.game_id, h.event, h.tournament_name, count(e.move) as number_of_moves FROM history_game_info h \
JOIN history_moves e ON h.game_id = e.game_id GROUP BY h.game_id, h.event, h.tournament_name ORDER BY count(e.move) DESC LIMIT 1)  \
UNION ALL (SELECT h.game_id, h.event, h.tournament_name, count(e.move) FROM history_game_info h \
JOIN history_moves e ON h.game_id = e.game_id \
GROUP BY h.game_id, h.event, h.tournament_name ORDER BY count(e.move) ASC LIMIT 1)")
longest_shortest.show()
longest_shortest.toPandas().to_csv("df5.csv")

+--------------------+--------------------+---------------+---------------+
|             game_id|               event|tournament_name|number_of_moves|
+--------------------+--------------------+---------------+---------------+
|4424a0a4-3732-407...|         FIDE WCh KO|  FideChamp2000|            291|
|1846cede-0037-4f0...|World Championshi...| WorldChamp1972|              1|
+--------------------+--------------------+---------------+---------------+



6. Shortest and Longest Draw game ever Played.
Result attributes: game_id, event, tournament_name, number_of_moves
Final result will have only two rows

In [15]:
draw = spark.sql("(SELECT h.game_id, h.event, h.tournament_name, count(e.move) as number_of_moves FROM history_game_info h JOIN history_moves e ON h.game_id = e.game_id WHERE h.result = '1/2-1/2' GROUP BY h.game_id, h.event, h.tournament_name ORDER BY count(e.move) DESC LIMIT 1) UNION ALL (SELECT h.game_id, h.event, h.tournament_name, count(e.move) as number_of_moves FROM history_game_info h JOIN history_moves e ON h.game_id = e.game_id WHERE h.result = '1/2-1/2' GROUP BY h.game_id, h.event, h.tournament_name ORDER BY count(e.move) ASC LIMIT 1)")
draw.show()
draw.toPandas().to_csv("df6.csv")
        

+--------------------+-------------+---------------+---------------+
|             game_id|        event|tournament_name|number_of_moves|
+--------------------+-------------+---------------+---------------+
|88f34084-e4df-490...|  FIDE WCh KO|  FideChamp2002|            258|
|a27c2b95-fb62-4b6...|FIDE-Wch k.o.|  FideChamp1998|             12|
+--------------------+-------------+---------------+---------------+



7. Most and Least rated Player. ------------------
Result attributes: player_name, elo
Chess Funda: elo is the rating of the player in chess tournament.
Final result will have only two rows

In [16]:
rating = spark.sql("(SELECT winner as player_name, winner_elo as elo FROM history_game_info WHERE winner_elo IS NOT NULL ORDER BY winner_elo DESC LIMIT 1) UNION ALL (SELECT winner as player_name, winner_elo as elo FROM history_game_info WHERE winner_elo IS NOT NULL ORDER BY winner_elo ASC LIMIT 1)")
rating.show()
rating.toPandas().to_csv("df7.csv")

+---------------+----+
|    player_name| elo|
+---------------+----+
|Carlsen, Magnus|2870|
|     Ciuksyte,D|2271|
+---------------+----+



In [17]:
# 8. 3rd Last Player with most Loss.
# Result attributes: player_name
# Final result will have only one row

#SELECT loser as player_name, count(loser) as number_of_losses FROM history_game_info WHERE result = '0-1' GROUP BY loser ORDER BY number_of_losses DESC

third_loss = spark.sql("SELECT loser as player_name, count(loser) as number_of_losses FROM history_game_info WHERE result = '0-1' GROUP BY loser ORDER BY number_of_losses DESC LIMIT 3")
third_loss.show()
third_loss.toPandas().to_csv("df8.csv")

+------------------+----------------+
|       player_name|number_of_losses|
+------------------+----------------+
|Botvinnik, Mikhail|              15|
| Steinitz, William|              14|
|         Euwe, Max|              11|
+------------------+----------------+



9. How many times players with low rating won matches with their total win Count.
Result attributes: player_name, win_count

In [18]:


low_rating = spark.sql("select winner, winner_elo from history_game_info where winner_elo is not null order by winner_elo asc limit 2")
low_rating.show()
low_rating.toPandas().to_csv("df9.csv")

+-------------+----------+
|       winner|winner_elo|
+-------------+----------+
|   Ciuksyte,D|      2271|
|Baginskaite,C|      2336|
+-------------+----------+



In [19]:
# 10. Move Sequence for Each Player in a Match.
# Result attributes: game_id, player_name, move_sequence, move_count

move_sequence = spark.sql("SELECT h.game_id, h.winner as player_name, concat_ws(' ', collect_list(e.move)) as move_sequence, count(e.move) as move_count FROM history_game_info h JOIN history_moves e ON h.game_id = e.game_id GROUP BY h.game_id, h.winner ORDER BY h.game_id, h.winner")
move_sequence.show()
move_sequence.toPandas().to_csv("df10.csv")

+--------------------+--------------------+--------------------+----------+
|             game_id|         player_name|       move_sequence|move_count|
+--------------------+--------------------+--------------------+----------+
|0028cc2e-5aaa-464...|                draw|d2d4 g8f6 g1f3 g7...|        33|
|00563036-1aa1-42c...|                draw|e2e4 c7c5 c2c3 g8...|        89|
|0059d5cb-2af8-48f...|                draw|d2d4 d7d5 c2c4 c7...|        62|
|0075db46-38dd-4ce...|                draw|e2e4 e7e6 d2d4 d7...|        76|
|007ba523-8617-42b...|Fischer, Robert J...|e2e4 g8f6 e4e5 f6...|       148|
|00bdc5cb-22b3-4fc...|       Polgar, Judit|d2d4 g8f6 c2c4 g7...|        82|
|00c5cb30-540b-47b...|                draw|d2d4 d7d5 c2c4 e7...|        58|
|00cb694b-245d-4a5...|                draw|c2c4 g8f6 b1c3 d7...|        97|
|00ff8f8a-33d1-466...| Tkachiev, Vladislav|d2d4 g8f6 g1f3 e7...|        49|
|01102870-dbe8-4fd...|         Kotronias,V|d2d4 g8f6 g2g3 g7...|       108|
|0128c0c8-d9

12. List All Perfect Tournament with Winner Name.
Chess Funda: Perfect Tournament means a player has won all the matches excluding draw matches. e.g Player A has won 5 matches out of 7
Matches in tournament where 2 matches are draw and player B has won 0 matches)
Result attributes: winner_name, tournament_name

In [20]:
perfect_tournament = df.select("game_id", "tournament_name", "event", "winner").where((df.winner != "draw") & (df.loser != "draw"))
perfect_tournament.show()
perfect_tournament.toPandas().to_csv("df12.csv")

+--------------------+---------------+------------------+------------------+
|             game_id|tournament_name|             event|            winner|
+--------------------+---------------+------------------+------------------+
|58e83255-93bb-4d5...| WorldChamp2021|          WCh 2021|         Carlsen,M|
|8a4ccd8c-d437-429...| WorldChamp2021|          WCh 2021|         Carlsen,M|
|55a122db-27d1-495...| WorldChamp2021|          WCh 2021|         Carlsen,M|
|c512a953-a341-47c...| WorldChamp2021|          WCh 2021|         Carlsen,M|
|8a49457a-7c11-436...| WorldChamp2016|          WCh 2016|  Karjakin, Sergey|
|dea461a0-46b1-43e...| WorldChamp2016|          WCh 2016|   Carlsen, Magnus|
|96cea9d3-9ded-432...| WorldChamp2016| WCh Rapid TB 2016|   Carlsen, Magnus|
|69954db6-8a4e-43d...| WorldChamp2016| WCh Rapid TB 2016|   Carlsen, Magnus|
|c6da46fc-4ed5-4e6...| WorldChamp2014|          WCh 2014|   Carlsen, Magnus|
|e8e1e55d-e0d2-467...| WorldChamp2014|          WCh 2014|Anand, Viswanathan|

13. Player with highest winning ratio.
Hint: Winning ratio: (Number of rounds won)/(Number of rounds played)
Result attributes: player_name
Final result will have only one row

14. Player who had given checkmate with Pawn. -------------------
Note: Consider all events for this query
Result attributes: player_name
Final result will have only one row

In [21]:
pawn_checkmate = df1.select("player").where((df1.piece == "P") & (df1.is_check_mate == 1))
pawn_checkmate.show()
pawn_checkmate.toPandas().to_csv("df14.csv")

+--------------+
|        player|
+--------------+
|Andersson, Ulf|
+--------------+



15. List games where player has won game without queen.
Result attributes: game_id, event, player_name

In [22]:
no_queen = df1.select(['game_id','player']).where((df1.white_queen_count == 0) &(df1.black_queen_count == 0) & (df1.is_check_mate == 1))
no_queen.show()
no_queen.toPandas().to_csv("df15.csv")

+--------------------+--------------+
|             game_id|        player|
+--------------------+--------------+
|04c81fe4-efd8-456...|    Nakamura,H|
|ac45a5ed-172d-4ae...|Andersson, Ulf|
+--------------------+--------------+

